# Face Recognition Model

In [ ]:
#Load and store the 4 values of train and validation data
import numpy as np
#import the data from the npz file and load it to trainng and validation 
dataset = np.load('/content/drive/My Drive/Project/facerecognition.npz')
print(dataset.files)
x_train = dataset['arr_0']
y_train = dataset['arr_1']
x_val = dataset['arr_2']
y_val = dataset['arr_3']

In [ ]:
#convert string labels to numbers
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
#convert the string labels to numbers
label_encoder= LabelEncoder()
y_train = LabelEncoder().fit_transform(y_train)
y_train = to_categorical(y_train,4)
y_val = label_encoder.fit_transform(y_val)
y_val = to_categorical(y_val,4)

In [ ]:
from keras.applications.vgg16 import VGG16,preprocess_input

#import VGG16 model which is the base model
vgg16 = VGG16(weights = 'imagenet', 
              input_shape = (224, 224, 3),
              include_top=False,
              pooling = None                  
)

for layer in vgg16.layers:
    layer.trainable = False 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#produce artificial data to expant x_train
datasetgen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,
        zca_epsilon=1e-06,  
        zca_whitening=False,  
        rotation_range=5,
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        shear_range=0.0,  
        zoom_range = 0.1, 
        brightness_range=None,
        horizontal_flip=True,  
        vertical_flip=False) 

datasetgen.fit(x_train)

In [ ]:
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import SGD

#create the model and layers
model = Sequential()
model.add(vgg16)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))
model.summary()

#compile the model
trained = model.compile(
    loss='categorical_crossentropy',
    optimizer= SGD(learning_rate = 1e-4, momentum=0.9, nesterov = False),
    metrics=['accuracy']
)


In [ ]:
#Draw the figure for the model
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#fit the model with the augmented data and 200 epochs
trained = model.fit_generator(datasetgen.flow(x_train, y_train, batch_size = 16),
    validation_data  = (x_val,y_val), validation_steps = 100, steps_per_epoch = len(x_train) // 16,
    epochs = 200)

In [ ]:
#save model as h5 to load in the application
model.save("rec_model.h5")

In [ ]:
#draw graphs for illustrating accuracy and loss
import matplotlib.pyplot as plt

plt.plot(trained.history['accuracy'])
plt.plot(trained.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

plt.plot(trained.history['loss'])
plt.plot(trained.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()